In [9]:
import psycopg2 as pg2
import pandas as pd
# pd.set_option('display.expand_frame_repr', False)
# pd.set_option('display.max_colwidth', -1)
import pprint 
import string
import unicodedata
import numpy as np
import pickle

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import decomposition

import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix

import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.util import ngrams
from nltk import pos_tag
from nltk import RegexpParser
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans

## Import pickled cleaned data from previous project

In [10]:
with open('../data/boardgamedata.pickle', 'rb') as f:
    worddata, numdata = pickle.load(f)
    
worddata.reset_index(drop=True, inplace=True)

In [11]:
worddata[10:15]

game.id              name                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

## Build LDA Model

In [12]:
#define common words in data to remove
#built from pervious project
sw = set(stopwords.words('english'))

COMMON_WORDS = ["game","point","board","player","rule","turn",
                "tile","card","deck","hand","points","victory",
                "win","lose","defeat","quot","players","play"
               ]
sw.update(COMMON_WORDS)

#### Remove numbers from vectorizer

Implement preprocess function

In [13]:
def preprocess_text(text):
    
#if text is empty, return empty bag of words
    if (text == None):
        return []

    if (len(text) < 1):
        return []
#remove numbers
    text = text.lower()
    text = re.sub(r'\d+', '', text)

#define common words in data to remove
    COMMON_WORDS = ["game","point","board","player","rule","turn",
                "tile","card","deck","hand","points","victory",
                "win","lose","defeat","quot","players","play"]
    
#removes common words
    for word in text.split():
        if word in COMMON_WORDS:
            text = text.replace(word, "")

#tokenize sentences and add part of speech tag
    sent_tokens = sent_tokenize(text)

    tokens = list(map(word_tokenize, sent_tokens))

    sent_tags = list(map(pos_tag, tokens))

    grammar = r"""
        SENT: {<(J|N).*>}    """

    cp = RegexpParser(grammar)
    ret_tokens = list()
    stemmer_snowball = SnowballStemmer('english')
    lemmatizer = WordNetLemmatizer()

    for sent in sent_tags:
        tree = cp.parse(sent)
        for subtree in tree.subtrees():
            if subtree.label() == 'SENT':
                t_tokenlist = [tpos[0].lower() for tpos in subtree.leaves()]
                
                t_tokens_lemmatize = list(map(lemmatizer.lemmatize, t_tokenlist))
                t_tokens_stemsnowball = list(map(stemmer_snowball.stem, t_tokens_lemmatize))
                ret_tokens.extend(t_tokens_stemsnowball)
    ret = []

#runs through the ret_tokens and compare with common words again and removes common words
    for token in ret_tokens:
        if token in COMMON_WORDS:
            pass
        else:
            ret.append(token)
    return(' '.join(ret))

In [15]:
count_vectorizer = CountVectorizer(stop_words=sw, 
                                   preprocessor=preprocess_text, 
                                   max_df=0.85,
                                  min_df=2, 
                                   max_features=1000,
                                  strip_accents='unicode')

In [16]:
word_vec = count_vectorizer.fit_transform(worddata['description'])

/home/zy/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ourselv', 'sha', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [19]:
lda = LatentDirichletAllocation(learning_method='online', n_jobs=-2, 
                                random_state=1659)
lda.fit(word_vec)

LatentDirichletAllocation(learning_method='online', n_jobs=-2,
                          random_state=1659)

In [21]:
def top_topic_features(model, feature_names, num_features=10):
    sorted_topics = feature_names[model.components_.argsort(axis=1)[:, ::-1][:, :num_features]]
    return sorted_topics

In [22]:
feature_names = np.array(count_vectorizer.get_feature_names())
top_topic_features(lda, feature_names=feature_names)


array([['ship', 'action', 'power', 'attack', 'resourc', 'phase', 'build',
        'control', 'star', 'world'],
       ['piec', 'color', 'space', 'squar', 'anim', 'red', 'pawn',
        'oppon', 'side', 'move'],
       ['dice', 'die', 'roll', 'number', 'space', 'track', 'first',
        'aacut', 'marker', 'move'],
       ['word', 'question', 'first', 'letter', 'answer', 'child', 'fun',
        'correct', 'object', 'space'],
       ['race', 'oppon', 'mission', 'car', 'eacut', 'element', 'creatur',
        'fast', 'differ', 'damag'],
       ['er', 'round', 'number', 'end', 'score', 'valu', 'face', 'tabl',
        'set', 'pile'],
       ['token', 'citi', 'hous', 'properti', 'monopoli', 'coin', 'set',
        'edit', 'symbol', 'alien'],
       ['team', 'hero', 'money', 'event', 'time', 'faction', 'compani',
        'room', 'master', 'market'],
       ['war', 'battl', 'unit', 'scenario', 'combat', 'armi', 'map',
        'forc', 'counter', 'system'],
       ['new', 'charact', 'expans', 'rsquo

### Create Probability Matrix

In [23]:
def predict_proba(model, vectorizer, text):
    if type(text) == str:
        text = [text]
    vec_text = vectorizer.transform(text)
    doc_probs = model.transform(vec_text)
    return doc_probs

In [24]:
game_probs = predict_proba(lda, count_vectorizer, worddata.description)

KeyboardInterrupt: 

In [19]:
#pickle the fitted transformed model

with open('../models/lda.pickle', 'wb') as f:
    pickle.dump([lda,count_vectorizer,game_probs], f)

## Building the Recommender

In [22]:
# function to find index of FIRST  game given the game title

def find_game_idx(df, game_title):
    try:
        return df.name[df.name.str.contains(game_title)].index[0]
    except:
        return None

def find_game_id(df, idx):
    return df['game.id'].iloc[idx]


## TODO: if search returns multiple results, what then?? Need to solve this

In [23]:
search = worddata.name.str.contains('Betrayal')
search.sum()


11

## /TODO

In [25]:
from sklearn.metrics.pairwise import cosine_distances, euclidean_distances

In [26]:
def sort_by_distance(idx, probs, distance_funct=cosine_distances, num_games=10):
    #returns array of indices of games closest to the provided game index
    return distance_funct(probs[idx].reshape(1,-1), probs).ravel().argsort()[:10]

* Debug sort_by_distance function
* check distances inside the function


In [27]:
def find_closest_games_names(sorted_distances, names):
    #finds names of an array of game idx's
    name_array = names.iloc[sorted_distances]
    return name_array

In [34]:
game_index = find_game_idx(worddata,'Catan')
game_sim = sort_by_distance(game_index, game_probs)
find_closest_games_names(game_sim,worddata.name)


12                                          Catan
54254                              Pocket Paddles
71292                       Dice City: Shipwright
62950                          Small Star Empires
68682          Dice Bazaar: Monkey Mini Expansion
26389                         El Juego de la Liga
132                             Buy Low Sell High
5131                      The Ark of the Covenant
63732    All About Town: Columbia, South Carolina
2013                                      Wembley
Name: name, dtype: object

In [28]:
def recommend(df, game_title, prob_matrix, 
             distance_funct=cosine_distances, num_games=10):
    game_index = find_game_idx(df, game_title)
    game_sim = sort_by_distance(game_index, prob_matrix, distance_funct, num_games)
    game_recs = find_closest_games_names(game_sim, df.name)
    game_ids = find_game_id(df, game_rec.index)
    
#     rec_df = pd.DataFrame({'id':game_ids,'name':game_recs})
    rec_df = find_closest_games(game_sim,df)
    
    return rec_df

In [29]:
game_rec = recommend(worddata, 'Catan', game_probs)
game_rec

NameError: name 'game_probs' is not defined

In [126]:
def find_closest_games(sorted_distances, gamedata_df):
    #finds names of an array of game idx's
    name_array = gamedata_df.iloc[sorted_distances]
    return name_array

In [ ]:
data = pd.read_csv('recs.csv')

In [61]:
#save prob matrix for use later
with open('../models/prob_matrix.pickle', 'wb') as f:
    pickle.dump(game_probs,f)

## TODO: use suprise Library to test

In [99]:
def load_data():
    with open('../data/boardgamedata.pickle', 'rb') as f:
        worddata, numdata = pickle.load(f)
    worddata.reset_index(drop=True, inplace=True)

    return worddata, numdata

def load_model():
    with open('../models/prob_matrix.pickle', 'rb') as f:
        game_probs = pickle.load(f)
    return game_probs

def find_game_idx(df, game_title):
    #returns the first found index of a game given a name search string query
    return df.name[df.name.str.contains(game_title)].index[0]


def find_game_id(df, idx):
    #returns game id of an array of indices of games
    return df['game.id'].iloc[idx]


def sort_by_distance(idx, probs, num_games):
    #returns array of indices of games most similiar to the provided game index
    return euclidean_distances(probs[idx].reshape(1,-1), probs).ravel().argsort()[:num_games]

def find_closest_games_names(sorted_distances, names):
    #finds names of an array of game idx's
    name_array = names.iloc[sorted_distances]
    return name_array

def find_closest_games(sorted_distances, gamedata_df):
    #finds names of an array of game idx's
    name_array = gamedata_df.iloc[sorted_distances]
    return name_array

def recommend_games(game_title, num_games):
    #returns recommendations for games in a list format
    df, numdata = load_data()
    prob_matrix = load_model()
    
    game_index = find_game_idx(df, game_title)
    game_sim = sort_by_distance(game_index, prob_matrix, num_games)
    game_recs = find_closest_games_names(game_sim, df.name)
    game_ids = find_game_id(df, game_recs.index)
    
    rec_df = find_closest_games(game_sim,df)
    to_html = rec_df.drop(axis=1,columns='game.id')
    to_html['category'] = to_html.category.apply(lambda x: ",".join(x))
    to_html['mechanic'] = to_html.mechanic.apply(lambda x: ",".join(x))
    to_html['short_desc'] = to_html.description.apply(lambda x: ' '.join(re.split(r'(?<=[.:;])\s', x)[:4]))
    to_html.drop(axis=1,columns='description',inplace=True)
    return to_html

In [100]:
data = recommend_games("Catan",10)

In [101]:
data.head()

,name,category,mechanic,short_desc
12,Catan,Negotiation,"Dice Rolling,Hand Management,Modular Board,Route/Network Building,Trading","In Catan (formerly The Settlers of Catan), players try to be the dominant force on the island of Catan by building settlements, cities, and roads. On each turn dice are rolled to determine what resources the island produces. Players collect these resources (cards)&mdash;wood, grain, brick, sheep, or stone&mdash;to build up their civilizations to get to 10 victory points and win the game.&#10;&#10;Setup includes randomly placing large hexagonal tiles (each showing a resource or the desert) in a honeycomb shape and surrounding them with water tiles, some of which contain ports of exchange. Number disks, which will correspond to die rolls (two 6-sided dice are used), are placed on each resource tile."
54254,Pocket Paddles,"Nautical,Print & Play,Racing","Auction/Bidding,Point to Point Movement","Pocket Paddles is a free print and play game for two players that fits in your wallet. All you need to play is the credit card-sized gameboard and 7 coins per player.&#10;&#10;In Pocket Paddles players compete in a furious paddle boat race. Whoever lands three ships on the sandy shores of Victor Island first, is declared winner, and is rewarded the platinum-power-paddle. The coins act as currency in a blind-bidding and bluffing round, in which the movement of your ships is determined."
71292,Dice City: Shipwright,"City Building,Expansion for Base-game","Dice Rolling,Take That,Worker Placement",A promo card that was part of the Dice Tower 2016 (Season 12) Kickstarter campaign.&#10;&#10;Its text says: &#10;Reduce the cost of the next Trade Ship you take by 2 resources of any type.&#10;&#10;
62950,Small Star Empires,"Science Fiction,Territory Building","Area Control,Area Influence,Area Enclosure,Grid Movement,Modular Board","Small Star Empires is a quick area control game for 2-4 players. In this game, players colonize the galaxy using their ships, which they move on a modular board containing hexagonal spaces (systems). The modular board is made up of seven different double-sided sector tiles, which allows for a different map and different experience each time you play the game.&#10;&#10;During a turn, a player must move one of their ships on the board. They can move the ship only in a straight line, as far away as they want, but they cannot go over systems controlled by other players."
132,Buy Low Sell High,Economic,Commodity Speculation,"Another of Reiner Knizia's fleet of games, this one is about making profits by buying and selling business stock. Players use cards to manipulate the price of three types of stocks (oil, technology, and retail), and the buying and selling of the stocks is pretty severely limited. Short and sweet, this game is both elegant and abstract. In a change from the original version, the three stocks have different levels of volatility."


In [71]:
data.category

12       [Negotiation]                                
54254    [Nautical, Print & Play, Racing]             
71292    [City Building, Expansion for Base-game]     
62950    [Science Fiction, Territory Building]        
132      [Economic]                                   
26389    [Sports]                                     
63732    [Educational]                                
2013     [Dice, Sports]                               
48640    [Economic]                                   
15459    [Economic, Space Exploration, Transportation]
Name: category, dtype: object

In [80]:
data['category'] = data.category.apply(lambda x: ",".join(x))
data['mechanic'] = data.mechanic.apply(lambda x: ",".join(x))

In [94]:
text = data.description.iloc[0]
' '.join(re.split(r'(?<=[.:;])\s', text)[:3])

'In Catan (formerly The Settlers of Catan), players try to be the dominant force on the island of Catan by building settlements, cities, and roads. On each turn dice are rolled to determine what resources the island produces. Players collect these resources (cards)&mdash;wood, grain, brick, sheep, or stone&mdash;to build up their civilizations to get to 10 victory points and win the game.&#10;&#10;Setup includes randomly placing large hexagonal tiles (each showing a resource or the desert) in a honeycomb shape and surrounding them with water tiles, some of which contain ports of exchange.'

In [95]:
data['short_desc'] = data.description.apply(lambda x: ' '.join(re.split(r'(?<=[.:;])\s', x)[:4]))

In [96]:
data.head()

,name,description,category,mechanic,short
12,Catan,"In Catan (formerly The Settlers of Catan), players try to be the dominant force on the island of Catan by building settlements, cities, and roads. On each turn dice are rolled to determine what resources the island produces. Players collect these resources (cards)&mdash;wood, grain, brick, sheep, or stone&mdash;to build up their civilizations to get to 10 victory points and win the game.&#10;&#10;Setup includes randomly placing large hexagonal tiles (each showing a resource or the desert) in a honeycomb shape and surrounding them with water tiles, some of which contain ports of exchange. Number disks, which will correspond to die rolls (two 6-sided dice are used), are placed on each resource tile. Each player is given two settlements (think: houses) and roads (sticks) which are, in turn, placed on intersections and borders of the resource tiles. Players collect a hand of resource cards based on which hex tiles their last-placed house is adjacent to. A robber pawn is placed on the desert tile.&#10;&#10;A turn consists of possibly playing a development card, rolling the dice, everyone (perhaps) collecting resource cards based on the roll and position of houses (or upgraded cities&mdash;think: hotels) unless a 7 is rolled, turning in resource cards (if possible and desired) for improvements, trading cards at a port, and trading resource cards with other players. If a 7 is rolled, the active player moves the robber to a new hex tile and steals resource cards from other players who have built structures adjacent to that tile.&#10;&#10;Points are accumulated by building settlements and cities, having the longest road and the largest army (from some of the development cards), and gathering certain development cards that simply award victory points. When a player has gathered 10 points (some of which may be held in secret), he announces his total and claims the win.&#10;&#10;Catan has won multiple awards and is one of the most popular games in recent history due to its amazing ability to appeal to experienced gamers as well as those new to the hobby.&#10;&#10;Die Siedler von Catan was originally published by KOSMOS and has gone through multiple editions. It was licensed by Mayfair and has undergone four editions as The Settlers of Catan. In 2015, it was formally renamed Catan to better represent itself as the core and base game of the Catan series. It has been re-published in two travel editions, portable edition and compact edition, as a special gallery edition (replaced in 2009 with a family edition), as an anniversary wooden edition, as a deluxe 3D collector's edition, in the basic Simply Catan, as a beginner version, and with an entirely new theme in Japan and Asia as Settlers of Catan: Rockman Edition. Numerous spin-offs and expansions have also been made for the game.&#10;&#10;",Negotiation,"Dice Rolling,Hand Management,Modular Board,Route/Network Building,Trading","In Catan (formerly The Settlers of Catan), players try to be the dominant force on the island of Catan by building settlements, cities, and roads. On each turn dice are rolled to determine what resources the island produces. Players collect these resources (cards)&mdash;wood, grain, brick, sheep, or stone&mdash;to build up their civilizations to get to 10 victory points and win the game.&#10;&#10;Setup includes randomly placing large hexagonal tiles (each showing a resource or the desert) in a honeycomb shape and surrounding them with water tiles, some of which contain ports of exchange. Number disks, which will correspond to die rolls (two 6-sided dice are used), are placed on each resource tile."
54254,Pocket Paddles,"Pocket Paddles is a free print and play game for two players that fits in your wallet. All you need to play is the credit card-sized gameboard and 7 coins per player.&#10;&#10;In Pocket Paddles players compete in a furious paddle boat race. Whoever lands three ships on the sandy shores of Victor Isla

In [102]:
data.values.tolist()

[['Catan',
  'Negotiation',
  'Dice Rolling,Hand Management,Modular Board,Route/Network Building,Trading',
  'In Catan (formerly The Settlers of Catan), players try to be the dominant force on the island of Catan by building settlements, cities, and roads. On each turn dice are rolled to determine what resources the island produces. Players collect these resources (cards)&mdash;wood, grain, brick, sheep, or stone&mdash;to build up their civilizations to get to 10 victory points and win the game.&#10;&#10;Setup includes randomly placing large hexagonal tiles (each showing a resource or the desert) in a honeycomb shape and surrounding them with water tiles, some of which contain ports of exchange. Number disks, which will correspond to die rolls (two 6-sided dice are used), are placed on each resource tile.'],
 ['Pocket Paddles',
  'Nautical,Print & Play,Racing',
  'Auction/Bidding,Point to Point Movement',
  'Pocket Paddles is a free print and play game for two players that fits in your